In [1]:
from datetime import datetime, timedelta
import pandas as pd
from statistics import mean

## Code below formats pax itineratary data for May 2nd

In [3]:
# Read in the pax itineratary data
pax_itinerary = pd.read_csv('Data/Large_Pax_Itineraries.csv')

# Remove the leading space in each of the column names
pax_itinerary.columns = pax_itinerary.columns.str.strip()

# Filter out the rows where 'month' is not 5 and 'day' is not 2
pax_itinerary = pax_itinerary[(pax_itinerary['month'] == 5) & (pax_itinerary['day_of_month'] == 2)]

# print first 5 rows
print(pax_itinerary.head())

# Save the dataframe as a CSV file
pax_itinerary.to_csv('Data/Pax_Itineraries_May_2nd.csv', index=False)

      year  quarter  month  day_of_month  planned_num_flights  \
5918  2016        2      5             2                    2   
5919  2016        2      5             2                    2   
5920  2016        2      5             2                    2   
5921  2016        2      5             2                    2   
5922  2016        2      5             2                    2   

      planned_multi_carrier  planned_first_flight_id  \
5918                      0                  1836205   
5919                      0                  1836205   
5920                      0                  1836205   
5921                      0                  1836205   
5922                      0                  1836205   

      planned_second_flight_id planned_first_carrier planned_second_carrier  \
5918                   1829041                    AA                     AA   
5919                   1829665                    AA                     AA   
5920                   1833100     

In [2]:
def time_to_minutes(time_str):
    if time_str is None:
        return None
    
    if time_str == "Inf":
        return float("inf")
    
    if isinstance(time_str, pd.Timestamp):
        time_str = time_str.strftime("%H%M")
    
    # Ensure the time_str is 4 characters long (e.g., "720" becomes "0720")
    time_str = time_str.zfill(4)
    
    hours = int(time_str[:2]) 
    minutes = int(time_str[2:])
    return hours * 60 + minutes


In [3]:
def get_formatted_time(time_str):
    if time_str is None or pd.isna(time_str):
        return None
    else:
        return time_str.strftime("%H%M")

## Cleaning flight and itinerary data to input into optimization model

In [7]:
# Read in the pax itineratary data
pax_itinerary = pd.read_csv('Data/Large_Pax_Itineraries.csv')

# Remove the leading space in each of the column names
pax_itinerary.columns = pax_itinerary.columns.str.strip()

# Filter out the rows where 'month' is not 5 and 'day' is not 2
pax_itinerary = pax_itinerary[(pax_itinerary['month'] == 5) & (pax_itinerary['day_of_month'] == 2)]

# Save the dataframe as a CSV file
pax_itinerary.to_csv('Data/Pax_Itineraries_May_2nd.csv', index=False)

# Load the data
file_path_flights = "Data/Sample_Day_May_2nd.txt"

df_flights = pd.read_csv(file_path_flights, delimiter='\t')
df_pax = pax_itinerary

# Filter so that there are only flights that departs from DFW dep terminals A, B, or C or arrive at DFW at arrival terminals A, B, or C
df_flights = df_flights[((df_flights['SchedDepApt'] == 'DFW') & (df_flights['DepGate'].str.startswith(('A','B','C'))) & (df_flights['DepGate'].notna())) 
                        | ((df_flights['SchedArrApt'] == 'DFW') & (df_flights['ArrGate'].str.startswith(('A','B','C'))) & (df_flights['ArrGate'].notna()))]

raw_df = df_flights

formatted_path = "Data/Formatted_Sample_Day_May_2nd.csv"
formatted_df = pd.read_csv(formatted_path)

raw_df['CombinedFlightNumber'] = raw_df['AirlineCode'].astype(str) + raw_df['FlightNumber'].astype(str)

merged_df = pd.merge(raw_df, formatted_df[['Flight Number', 'Aircraft', 'Flight Type']], 
                left_on='CombinedFlightNumber', right_on='Flight Number', how='left')

# Now, propagate the 'Aircraft' values across rows with matching 'TailNumber'
merged_df['Aircraft'] = merged_df.groupby('TailNumber')['Aircraft'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))

merged_df['SchedDepLocal'] = pd.to_datetime(merged_df['SchedDepLocal'], format="%Y-%m-%d %H:%M")
merged_df['SchedArrLocal'] = pd.to_datetime(merged_df['SchedArrLocal'], format="%Y-%m-%d %H:%M")
merged_df['InGateLocal'] = pd.to_datetime(merged_df['InGateLocal'], format="%Y-%m-%d %H:%M")
merged_df['OutGateLocal'] = pd.to_datetime(merged_df['OutGateLocal'], format="%Y-%m-%d %H:%M")

new_formatted_data = []
for _, row in merged_df.iterrows():
    is_departing = row['SchedDepApt'] == 'DFW'
    flight_number = row['CombinedFlightNumber']
    destination = row['SchedArrApt']
    origin = row['SchedDepApt']
    aircraft = row['Aircraft']
    flight_type = row['Flight Type']
    if is_departing:
        DepGate = row['DepGate']
        ArrGate = ""
    else:
        DepGate = ""
        ArrGate = row['ArrGate']

    # Check if the arrival time of the aircraft was within 2 hours, else assign it to 90 min before departure
    if is_departing:
        off_time = get_formatted_time(row['SchedDepLocal'])
        off_time_actual = get_formatted_time(row['OutGateLocal'])

        arrival_row = merged_df[(merged_df['TailNumber'] == row['TailNumber']) &
                                (merged_df['SchedArrApt'] == 'DFW') &
                                (row['SchedDepLocal'] - merged_df['SchedArrLocal'] <= timedelta(hours=2)) &
                                (merged_df['SchedArrLocal'] < row['SchedDepLocal'])]
        arrival_row = arrival_row.sort_values(by='SchedArrLocal', ascending=False)

        if not arrival_row.empty:
            arrival_time = get_formatted_time(arrival_row.iloc[0]['SchedArrLocal'])
        else:
            arrival_time = get_formatted_time(row['SchedDepLocal'] - timedelta(minutes=90))

        arrival_time_actual_row = merged_df[(merged_df['TailNumber'] == row['TailNumber']) &
                                            (merged_df['SchedArrApt'] == 'DFW') &
                                            (row['SchedDepLocal'] - merged_df['InGateLocal'] <= timedelta(hours=2)) &
                                            (merged_df['InGateLocal'] < row['SchedDepLocal'])]
        arrival_time_actual_row = arrival_time_actual_row.sort_values(by='InGateLocal', ascending=False)

        if not arrival_time_actual_row.empty:
            arrival_time_actual = get_formatted_time(arrival_time_actual_row.iloc[0]['InGateLocal'])
        else:
            arrival_time_actual = get_formatted_time(row['SchedDepLocal'] - timedelta(minutes=90))

        passengers_arr = 0  # No arriving passengers in departing flights
        passengers_dept = {1: 300, 2: 150, 3: 75}.get(aircraft, 0)

    #If arriving flight, check if the aircraft is departing within the next 2 hours, else assign 30 min after arrival
    else:
        arrival_time = get_formatted_time(row['SchedArrLocal'])
        arrival_time_actual = get_formatted_time(row['InGateLocal'])

        off_row = merged_df[(merged_df['TailNumber'] == row['TailNumber']) &
                                (merged_df['SchedDepApt'] == 'DFW') &
                                (merged_df['SchedDepLocal'] - row['SchedArrLocal'] <= timedelta(hours=2)) &
                                (merged_df['SchedDepLocal'] > row['SchedArrLocal'])]
        off_row = off_row.sort_values(by='SchedDepLocal', ascending=True)

        if not off_row.empty:
            off_time = get_formatted_time(off_row.iloc[0]['SchedDepLocal'])
        else:
            off_time = get_formatted_time(row['SchedArrLocal'] + timedelta(minutes=30))

        off_time_actual_row = merged_df[(merged_df['TailNumber'] == row['TailNumber']) &
                                            (merged_df['SchedDepApt'] == 'DFW') &
                                            (merged_df['OutGateLocal'] - row['SchedArrLocal'] <= timedelta(hours=4)) &
                                            (merged_df['OutGateLocal'] > row['SchedArrLocal'])]
        off_time_actual_row = off_time_actual_row.sort_values(by='OutGateLocal', ascending=True)

        if not off_time_actual_row.empty:
            off_time_actual = get_formatted_time(off_time_actual_row.iloc[0]['OutGateLocal'])
        else:
            off_time_actual = get_formatted_time(row['SchedArrLocal'] + timedelta(minutes=30))

        passengers_arr = {1: 300, 2: 150, 3: 75}.get(aircraft, 0)
        passengers_dept = 0         # No departing passengers in arriving flights

    new_formatted_data.append({
        #"FlightIndex": row['flight_index'],
        "FlightNumber": flight_number,
        "IsDeparting": 'Y' if is_departing else 'N',
        "Destination": destination,
        "Origin": origin,
        "ArrivalTime": arrival_time,
        "OffTime": off_time,
        "ArrivalTimeMinutes": time_to_minutes(arrival_time),
        "OffTimeMinutes": time_to_minutes(off_time),
        "ArrivalTimeActual": arrival_time_actual,
        "OffTimeActual": off_time_actual,
        "ArrivalTimeActualMinutes": time_to_minutes(arrival_time_actual),
        "OffTimeActualMinutes": time_to_minutes(off_time_actual),
        "Aircraft": aircraft,
        "FlightType": flight_type,
        "PassengersArr": passengers_arr,
        "PassengersDept": passengers_dept,
        "DepGate": DepGate,
        "ArrGate": ArrGate,
        "SchedDepLocal": row['SchedDepLocal'],
        "SchedArrLocal": row['SchedArrLocal'],
        "OutGateLocal": row['OutGateLocal'],
        "InGateLocal": row['InGateLocal'],
        "SchedDepUtc": row['SchedDepUtc'],
        "SchedArrUtc": row['SchedArrUtc'],
        "OutGateUtc": row['OutGateUtc'],
        "InGateUtc": row['InGateUtc'],
        "TailNumber": row['TailNumber'],
        #"num_connecting_pax": row['num_connecting_pax']
    })

# if the departure time is less than the arrival time, set the departure time to the arrival time + 24 hours
for row in new_formatted_data:
    if row['OffTimeMinutes'] is not None and row['ArrivalTimeMinutes'] is not None and row['OffTimeMinutes'] < row['ArrivalTimeMinutes']:
        row['OffTimeMinutes'] = row['OffTimeMinutes'] + 24 * 60

# if the actual departure time is less than the actual arrival time, set the actual departure time to the actual arrival time + 24 hours
for row in new_formatted_data:
    if row['OffTimeActualMinutes'] is not None and row['ArrivalTimeActualMinutes'] is not None and row['OffTimeActualMinutes'] < row['ArrivalTimeActualMinutes']:
        row['OffTimeActualMinutes'] = row['OffTimeActualMinutes'] + 24 * 60

new_formatted_df = pd.DataFrame(new_formatted_data)

# Clip PassengersDept and PassengersArr to 0
new_formatted_df['PassengersDept'] = new_formatted_df['PassengersDept'].clip(lower=0)
new_formatted_df['PassengersArr'] = new_formatted_df['PassengersArr'].clip(lower=0)

df_flights = new_formatted_df

# ---------------------------------
# New construction zone | df_flights needs to be new_formatted_df
# ---------------------------------

# Sort df_flights by ArrivalTimeMinutes
df_flights = df_flights.sort_values(by='ArrivalTimeMinutes')

# Initialize the connecting passengers column to 0
df_flights['num_connecting_pax'] = 0

# Create a flight index
df_flights['flight_index'] = range(1, len(df_flights) + 1)

# Create 'inbound flight index' and 'outbound flight index' columns in df_pax
df_pax['inbound_flight_index'] = None
df_pax['outbound_flight_index'] = None

# Initialize the connections matrix and connections tier matrix
connections_matrix = pd.DataFrame(0, index=df_flights['flight_index'], columns=df_flights['flight_index'])
connections_tier_matrix = pd.DataFrame(0, index=df_flights['flight_index'], columns=df_flights['flight_index'])

# Convert time columns to datetime
df_flights['SchedDepUtc'] = pd.to_datetime(df_flights['SchedDepUtc'], errors='coerce').dt.hour * 60 + pd.to_datetime(df_flights['SchedDepUtc'], errors='coerce').dt.minute
df_flights['SchedArrUtc'] = pd.to_datetime(df_flights['SchedArrUtc'], errors='coerce').dt.hour * 60 + pd.to_datetime(df_flights['SchedArrUtc'], errors='coerce').dt.minute
df_pax['planned_connection_time'] = pd.to_datetime(df_pax['planned_connection_time'], errors='coerce').dt.hour * 60 + pd.to_datetime(df_pax['planned_connection_time'], errors='coerce').dt.minute
df_pax['planned_departure_time'] = pd.to_datetime(df_pax['planned_departure_time'], errors='coerce').dt.hour * 60 + pd.to_datetime(df_pax['planned_departure_time'], errors='coerce').dt.minute

# Define time tolerance for matching (5 minutes)
time_tolerance = 5

# Find inbound and outbound flight indices for each itinerary
for flight_idx, flight_row in df_flights.iterrows():
    if flight_row['Origin'] == "DFW": # If departing
        pax_rows = df_pax[(df_pax['planned_destination'] == flight_row['Destination']) & # The itins with final destination the same as the destination of the flight that leaves DFW
                            (abs(df_pax['planned_connection_time'] - flight_row['SchedDepUtc']) <= time_tolerance)] # And the second leg departure time within 5 min of flight_row's departure time
        # pax_rows are the itineraries that have flight_row as the outbound leg
        if not pax_rows.empty:
            df_pax.loc[pax_rows.index, 'outbound_flight_index'] = flight_row['flight_index'] # Add this flight's index as the outbound leg flight index in itinerary df
            df_flights.at[flight_idx, 'num_connecting_pax'] = pax_rows['num_passengers'].sum() # Sum up the # of pax in the itineraries that have flight_row as the outbound leg

    elif flight_row['Destination'] == "DFW": # If arriving
        pax_rows = df_pax[(df_pax['planned_origin'] == flight_row['Origin']) & 
                            (abs(df_pax['planned_departure_time'] - flight_row['SchedDepUtc']) <= time_tolerance)]
        # pax_rows are the itineraries that have flight_row as the inbound leg
        if not pax_rows.empty:
            df_pax.loc[pax_rows.index, 'inbound_flight_index'] = flight_row['flight_index']
            df_flights.at[flight_idx, 'num_connecting_pax'] = pax_rows['num_passengers'].sum()

# Making an assumption that we count passengers on a 2-flight itinerary as connecting even if we only found one of the two flights
# So the total passengers in connections matrix will be fewer than the sum of num_connecting_pax in df_flights

# Populate the connections matrix
for _, itinerary_row in df_pax.iterrows():
    if itinerary_row['inbound_flight_index'] is not None and itinerary_row['outbound_flight_index'] is not None:
        connections_matrix.at[itinerary_row['inbound_flight_index'], itinerary_row['outbound_flight_index']] += itinerary_row['num_passengers']

# Populate the connection tier matrix
for _, itinerary_row in df_pax.iterrows():
    if itinerary_row['inbound_flight_index'] is not None and itinerary_row['outbound_flight_index'] is not None:
        if ((itinerary_row['planned_layover_duration'] > 0) and (itinerary_row['planned_layover_duration'] <= 45)):
            connections_tier_matrix.at[itinerary_row['inbound_flight_index'], itinerary_row['outbound_flight_index']] = 1
        elif ((itinerary_row['planned_layover_duration'] > 45) and (itinerary_row['planned_layover_duration'] <= 90)):
            connections_tier_matrix.at[itinerary_row['inbound_flight_index'], itinerary_row['outbound_flight_index']] = 2
            print(itinerary_row['planned_layover_duration'])
        else:
            connections_tier_matrix.at[itinerary_row['inbound_flight_index'], itinerary_row['outbound_flight_index']] = 3

# Save the updated connections matrix and connection tier matrix
connections_matrix.to_csv('Data/connections_matrix.csv')
small_connections_matrix = connections_matrix.iloc[0:100, 0:100]
small_connections_matrix.to_csv('Data/small_connections_matrix.csv')
connections_tier_matrix.to_csv('Data/connections_tier_matrix.csv')

df_flights.loc[df_flights['IsDeparting'] == 'Y', 'PassengersDept'] -= df_flights['num_connecting_pax']
df_flights.loc[df_flights['IsDeparting'] != 'Y', 'PassengersArr'] -= df_flights['num_connecting_pax']

#-------------------------
# End new construction zone
#-------------------------

df_flights.to_csv("Data/Final_Formatted_Sample_Day.csv", index=False)

# Create df which is first 100 rows of new_formatted_df
df_flights_small = df_flights.head(100)

# save to csv
df_flights_small.to_csv("Data/Small_Final_Formatted_Sample_Day.csv", index=False)

# save df_pax to csv
df_pax.to_csv("Data/Pax_Itineraries_May_2nd.csv", index=False)


/var/folders/6f/0t_pqxjx06q6_18yc4lp8j3w0000gn/T/ipykernel_1369/372973348.py:34: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_df['Aircraft'] = merged_df.groupby('TailNumber')['Aircraft'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))
/var/folders/6f/0t_pqxjx06q6_18yc4lp8j3w0000gn/T/ipykernel_1369/372973348.py:190: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_pax['planned_connection_time'] = pd.to_datetime(df_pax['planned_connection_time'], errors='coerce').dt.hour * 60 + pd.to_datetime(df_pax['planned_connection_time'], errors='coerce').dt.minute
/var/folders/6f/0t_pqxjx06q6_18yc4lp8j3w0000gn/T/ipykernel_1369/372973348.py:190: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dat

85
75
60
55
80
80
65
55
65
80
70
65
70
54
60
60
55
65
70
90
90
75
74
75
75
75
75
55
75
90
69
54
79
64
54
74
64
54
49
54
69
69
64
69
64
64
59
69
59
59
61
46
71
56
66
46
46
51
61
61
61
56
61
51
70
75
90
70
65
65
70
75
75
60
70
90
65
65
68
68
58
48
53
58
63
78
53
63
58
53
68
88
63
73
83
78
63
63
68
58
73
53
63
58
63
78
68
53
58
63
63
83
48
58
78
53
53
59
59
54
59
69
59
59
89
49
70
85
75
55
65
85
46
51
51
46
51
66
71
81
86
61
51
46
51
56
71
71
46
56
46
61
51
66
71
56
61
71
86
86
49
64
69
54
49
54
49
59
59
55
65
50
65
50
50
70
65
70
60
65
65
80
85
60
80
85
75
60
80
85
73
58
88
83
83
68
63
63
63
68
68
83
88
51
56
76
46
46
51
56
56
56
76
51
81
56
76
76
76
71
81
60
61
61
61
61
61
56
66
56
59
54
84
90
85
55
60
55
50
60
55
65
90
90
90
55
55
55
85
78
78
61
58
83
68
83
58
73
63
83
57
58
63
68
76
61
86
71
61
81
71
61
56
61
76
76
76
71
81
71
66
76
71
66
66
85
90
85
85
85
80
75
85
85
85
85
85
55
65
80
80
70
60
70
75
90
90
65
75
65
80
70
85
90
90
55
75
50
55
60
65
55
50
49
55
50
70
55
55
70
75
50
64
4

## Walking distance metrics based on actual 2016 gate assignments

In [20]:
print("\nMetrics based on actual 2016 gate assignments:")

num_flights = 31

# Read in the flight data and walking distances for arriving and departing passengers
file_path = "Data/Final_Formatted_Sample_Day.csv"
df = pd.read_csv(file_path)
df = df.iloc[:num_flights]
walking_distances = pd.read_csv("Data/Walking Distances Arriving and Departing Pax.csv")
print(walking_distances.columns)

# Filter df to include only rows with valid gate assignments based on walking distances
orig_rows = len(df)
valid_gates = set(walking_distances["Gate_Name"])
valid_indices = df.index[df["DepGate"].isin(valid_gates) | df["ArrGate"].isin(valid_gates)].tolist()
df = df.loc[valid_indices]
filtered_out = orig_rows - len(df)
print("Rows filtered out because gate assignment was not in terminals A, B, or C:", filtered_out)
print("Rows remaining in df:", len(df))
print("Indices of filtered out rows:", [i for i in range(orig_rows) if i not in valid_indices])

# Initialize accumulators for departing and arriving walking distances and passenger counts
total_departing_wd = 0.0
total_arriving_wd = 0.0
total_departing_passengers = 0
total_arriving_passengers = 0

# Process each flight row for departing and arriving distances
for _, row in df.iterrows():
    if row["IsDeparting"] == "Y":
        # Use DepGate for departing flights
        gate = row["DepGate"]
        gate_match = walking_distances[ walking_distances["Gate_Name"] == gate ]
        if gate_match.empty:
            print(f"WARNING: Departing gate {gate} not found in walking_distances.")
            continue
        distance = gate_match.iloc[0]["TSA_to_Gate"]
        total_departing_wd += row["PassengersDept"] * distance
        total_departing_passengers += row["PassengersDept"]
    else:
        # Use ArrGate for arriving flights
        gate = row["ArrGate"]
        gate_match = walking_distances[ walking_distances["Gate_Name"] == gate ]
        if gate_match.empty:
            print(f"WARNING: Arriving gate {gate} not found in walking_distances.")
            continue
        distance = gate_match.iloc[0]["Gate_to_Bag"]
        total_arriving_wd += row["PassengersArr"] * distance
        total_arriving_passengers += row["PassengersArr"]

avg_departing_wd = total_departing_wd / total_departing_passengers if total_departing_passengers else 0
avg_arriving_wd = total_arriving_wd / total_arriving_passengers if total_arriving_passengers else 0

# Load the connections matrices and convert to a NumPy array for easier indexing
conn_mat = pd.read_csv("Data/small_connections_matrix.csv", header=None)
conn_tier_mat = pd.read_csv("Data/connections_tier_matrix.csv", header=None)

# Filter to just be first num_flights rows and columns (plus remove the flight index row and column)
conn_mat = conn_mat.iloc[1:num_flights+1, 1:num_flights+1].to_numpy()  # Convert to numpy array
conn_tier_mat = conn_tier_mat.iloc[1:num_flights+1, 1:num_flights+1].to_numpy()

total_connection_wd_tier1 = 0.0
total_connection_passengers_tier1 = 0
total_connection_wd_tier2 = 0.0
total_connection_passengers_tier2 = 0
total_connection_wd_tier3 = 0.0
total_connection_passengers_tier3 = 0

# Load walking distances between gates.
# This CSV is expected to have a "Gate_Name" column and other columns with gate names.
walking_distances_gate_to_gate = pd.read_csv("Data/Walking Distances Gate-to-Gate.csv")

# Loop over flight pairs
num_rows, num_cols = conn_mat.shape
for i in range(0, num_rows):
    for j in range(0, num_cols):
        num_connect = conn_mat[i, j]
        if num_connect > 0:
            # Get tier of these connecting passengers
            tier = conn_tier_mat[i, j]

            # Retrieve flight rows corresponding to the current indices
            flight_i = df.iloc[i]
            flight_j = df.iloc[j]
            
            # Determine gate assignment based on flight type
            gate_i = flight_i["ArrGate"] # Flight i is always inbound
            gate_i_int_match = walking_distances[walking_distances["Gate_Name"] == gate_i]
            gate_i = gate_i_int_match.iloc[0]["Gate_Int"]

            gate_j = flight_j["DepGate"] # Flight j is always outbound
            gate_j_int_match = walking_distances[walking_distances["Gate_Name"] == gate_j]
            gate_j = gate_j_int_match.iloc[0]["Gate_Int"]
            
            gate_to_gate_wd = walking_distances_gate_to_gate.iloc[gate_i-1, gate_j-1]

            if tier == 1:
                total_connection_wd_tier1 += num_connect * gate_to_gate_wd
                total_connection_passengers_tier1 += num_connect
            
            if tier == 2:
                total_connection_wd_tier2 += num_connect * gate_to_gate_wd
                total_connection_passengers_tier2 += num_connect

            if tier == 3:
                total_connection_wd_tier3 += num_connect * gate_to_gate_wd
                total_connection_passengers_tier3 += num_connect

avg_connection_wd_tier1 = total_connection_wd_tier1 / total_connection_passengers_tier1
avg_connection_wd_tier2 = total_connection_wd_tier2 / total_connection_passengers_tier2
avg_connection_wd_tier3 = total_connection_wd_tier3 / total_connection_passengers_tier3

print("Average Departing Passenger Walking Distance:", round(avg_departing_wd, 2))
print("Average Arriving Passenger Walking Distance:", round(avg_arriving_wd, 2))
print("Average Tier 1 Connecting Passenger Walking Distance:", round(avg_connection_wd_tier1, 2))
print("Average Tier 2 Connecting Passenger Walking Distance:", round(avg_connection_wd_tier2, 2))
print("Average Tier 3 Connecting Passenger Walking Distance:", round(avg_connection_wd_tier3, 2))
print("Total flights:", num_flights)
print("Total Departing Passengers:", total_departing_passengers)
print("Total Arriving Passengers:", total_arriving_passengers)
print("Total Tier 1 Connecting Passengers:", total_connection_passengers_tier1)
print("Total Tier 2 Connecting Passengers:", total_connection_passengers_tier2)
print("Total Tier 3 Connecting Passengers:", total_connection_passengers_tier3)


Metrics based on actual 2016 gate assignments:
Index(['Gate_Int', 'Gate_Name', 'TSA_to_Gate', 'Gate_to_Bag'], dtype='object')
Rows filtered out because gate assignment was not in terminals A, B, or C: 0
Rows remaining in df: 31
Indices of filtered out rows: []
Average Departing Passenger Walking Distance: 70.99
Average Arriving Passenger Walking Distance: 136.02
Average Tier 1 Connecting Passenger Walking Distance: 1116.0
Average Tier 2 Connecting Passenger Walking Distance: 859.74
Average Tier 3 Connecting Passenger Walking Distance: 362.75
Total flights: 31
Total Departing Passengers: 986
Total Arriving Passengers: 1497
Total Tier 1 Connecting Passengers: 1
Total Tier 2 Connecting Passengers: 23
Total Tier 3 Connecting Passengers: 4


/var/folders/6f/0t_pqxjx06q6_18yc4lp8j3w0000gn/T/ipykernel_60548/1946294805.py:56: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  conn_tier_mat = pd.read_csv("Data/connections_tier_matrix.csv", header=None)


## Extracting walking distances based on optimized gate assignments

In [34]:
import pandas as pd
import numpy as np

def get_walking_distances(file_path = "Optimized_Gate_Assignments_Sample_Day.csv", actual_or_optimized = "optimized"):
    if actual_or_optimized == "actual":
        DepGate = "DepGate"
        ArrGate = "ArrGate"
    else:
        DepGate = "OptDepGate"
        ArrGate = "OptArrGate"

    print("\nMetrics based on optimized gate assignments:\n")

    df = pd.read_csv(file_path)
    num_flights = df.shape[0]

    # Load the walking distances for arriving and departing flights
    walking_distances = pd.read_csv("Data/Walking Distances Arriving and Departing Pax.csv")

    # Initialize accumulators for departing and arriving walking distances and passenger counts
    total_departing_wd = 0.0
    total_arriving_wd = 0.0
    total_departing_passengers = 0
    total_arriving_passengers = 0

    # Process each flight row for departing/arriving
    for _, row in df.iterrows():
        if row["IsDeparting"] == "Y":
            # Use DepGate for departing flights
            gate = row[DepGate]
            if pd.isna(gate):
                # Gate assignment is missing; skip or warn as needed
                print(f"WARNING: Departing gate assignment is missing for flight row:\n{row}")
                continue

            gate_match = walking_distances[walking_distances["Gate_Name"] == gate]
            if gate_match.empty:
                print(f"WARNING: Departing gate '{gate}' not found in walking_distances.")
                continue

            distance = gate_match.iloc[0]["TSA_to_Gate"]
            total_departing_wd += row["PassengersDept"] * distance
            total_departing_passengers += row["PassengersDept"]

        else:
            # Use ArrGate for arriving flights
            gate = row[ArrGate]
            if pd.isna(gate):
                # Gate assignment is missing; skip or warn as needed
                print(f"WARNING: Arriving gate assignment is missing for flight row:\n{row}")
                continue

            gate_match = walking_distances[walking_distances["Gate_Name"] == gate]
            if gate_match.empty:
                print(f"WARNING: Arriving gate '{gate}' not found in walking_distances.")
                continue

            distance = gate_match.iloc[0]["Gate_to_Bag"]
            total_arriving_wd += row["PassengersArr"] * distance
            total_arriving_passengers += row["PassengersArr"]

    # Compute averages
    avg_departing_wd = (
        total_departing_wd / total_departing_passengers
        if total_departing_passengers > 0
        else 0
    )
    avg_arriving_wd = (
        total_arriving_wd / total_arriving_passengers
        if total_arriving_passengers > 0
        else 0
    )

    # ----- CONNECTIONS -----
    # Load the connections matrix (adjusting indices).
    conn_mat = pd.read_csv("Data/connections_matrix.csv", header=None)
    # Slice out the relevant portion for your flights
    conn_mat = conn_mat.iloc[1:num_flights+1, 1:num_flights+1].to_numpy()

    total_connection_wd = 0.0
    total_connection_passengers = 0

    # Load gate-to-gate distances
    walking_distances_gate_to_gate = pd.read_csv("Data/Walking Distances Gate-to-Gate.csv")

    # For referencing gate "integers", we use the same walking_distances DataFrame,
    # which presumably has columns ["Gate_Name", "Gate_Int", "TSA_to_Gate", "Gate_to_Bag"]
    # We'll build a dictionary mapping gate name -> integer for faster lookups:
    gate_to_int = dict()
    for idx, row_w in walking_distances.iterrows():
        gate_name = row_w["Gate_Name"]
        gate_int  = int(row_w["Gate_Int"])  # or row_w["Gate_Int"]
        gate_to_int[gate_name] = gate_int

    # Loop over flight pairs in the connection matrix
    num_rows, num_cols = conn_mat.shape

    for i in range(num_rows):
        for j in range(num_cols):
            num_connect = conn_mat[i, j]
            if num_connect <= 0:
                continue  # no connecting passengers

            # Grab flight rows i (inbound) and j (outbound)
            try:
                flight_i = df.iloc[i]
                flight_j = df.iloc[j]
            except IndexError:
                # Just in case i or j is out of range for df
                print(f"WARNING: Index out of range when accessing flight {i} or {j}")
                continue

            # flight_i is the inbound flight => use ArrGate
            gate_i_name = flight_i[ArrGate]
            # flight_j is the outbound flight => use DepGate
            gate_j_name = flight_j[DepGate]

            # If either is missing, skip
            if pd.isna(gate_i_name) or pd.isna(gate_j_name):
                continue

            # Check if the name is in the gate_to_int dict
            if gate_i_name not in gate_to_int:
                print(f"WARNING: Arriving gate '{gate_i_name}' not found in gate_to_int dictionary.")
                continue
            if gate_j_name not in gate_to_int:
                print(f"WARNING: Departing gate '{gate_j_name}' not found in gate_to_int dictionary.")
                continue

            gate_i_int = gate_to_int[gate_i_name]
            gate_j_int = gate_to_int[gate_j_name]

            # gate-to-gate matrix is presumably 0-indexed, so we do [gate_i_int-1, gate_j_int-1]
            # but only if gate_i_int and gate_j_int are within the matrix bounds
            if not (1 <= gate_i_int <= walking_distances_gate_to_gate.shape[0]):
                print(f"WARNING: gate_i_int={gate_i_int} out of range for gate-to-gate matrix.")
                continue
            if not (1 <= gate_j_int <= walking_distances_gate_to_gate.shape[1]):
                print(f"WARNING: gate_j_int={gate_j_int} out of range for gate-to-gate matrix.")
                continue

            gate_to_gate_wd = walking_distances_gate_to_gate.iloc[gate_i_int-1, gate_j_int-1]

            # Add to total
            total_connection_wd += num_connect * gate_to_gate_wd
            total_connection_passengers += num_connect

    # Compute average connection distance
    avg_connection_wd = (
        total_connection_wd / total_connection_passengers
        if total_connection_passengers > 0
        else 0
    )

    print("Average Departing Passenger Walking Distance:", round(avg_departing_wd, 2))
    print("Average Arriving  Passenger Walking Distance:", round(avg_arriving_wd, 2))
    print("Average Connecting Passenger Walking Distance:", round(avg_connection_wd, 2))
    print("Total flights:", num_flights)
    print("Total Departing Passengers:", total_departing_passengers)
    print("Total Arriving Passengers:", total_arriving_passengers)
    print("Total Connecting Passengers:", total_connection_passengers)


In [35]:
get_walking_distances("Optimized_Gate_Assignments_Sample_Day.csv", "optimized")


Metrics based on optimized gate assignments:

Average Departing Passenger Walking Distance: 126.89
Average Arriving  Passenger Walking Distance: 110.7
Average Connecting Passenger Walking Distance: 0
Total flights: 31
Total Departing Passengers: 3115
Total Arriving Passengers: 406
Total Connecting Passengers: 0


/var/folders/6f/0t_pqxjx06q6_18yc4lp8j3w0000gn/T/ipykernel_98329/1635676756.py:76: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  conn_mat = pd.read_csv("Data/connections_matrix.csv", header=None)


## Now extract walking distances with tiered optimization

In [26]:
import pandas as pd
print("Metrics based on tiered optimized gate assignments:\n")

# Read in the flight data and walking distances for arriving and departing passengers
file_path = "Optimized_Gate_Assignments_Sample_Day.csv"
df = pd.read_csv(file_path)
num_flights = df.shape[0]
walking_distances = pd.read_csv("Data/Walking Distances Arriving and Departing Pax.csv")

# Initialize accumulators for departing and arriving walking distances and passenger counts
total_departing_wd = 0.0
total_arriving_wd = 0.0
total_departing_passengers = 0
total_arriving_passengers = 0

# Process each flight row for departing and arriving distances
for _, row in df.iterrows():
    if row["IsDeparting"] == "Y":
        # Use DepGate for departing flights
        gate = row["OptDepGate"]
        gate_match = walking_distances[ walking_distances["Gate_Name"] == gate ]
        if gate_match.empty:
            print(f"WARNING: Departing gate {gate} not found in walking_distances.")
            continue
        distance = gate_match.iloc[0]["TSA_to_Gate"]
        total_departing_wd += row["PassengersDept"] * distance
        total_departing_passengers += row["PassengersDept"]
    else:
        # Use ArrGate for arriving flights
        gate = row["OptArrGate"]
        gate_match = walking_distances[ walking_distances["Gate_Name"] == gate ]
        if gate_match.empty:
            print(f"WARNING: Arriving gate {gate} not found in walking_distances.")
            continue
        distance = gate_match.iloc[0]["Gate_to_Bag"]
        total_arriving_wd += row["PassengersArr"] * distance
        total_arriving_passengers += row["PassengersArr"]

avg_departing_wd = total_departing_wd / total_departing_passengers if total_departing_passengers else 0
avg_arriving_wd = total_arriving_wd / total_arriving_passengers if total_arriving_passengers else 0

# Load the connections matrices and convert to a NumPy array for easier indexing
conn_mat = pd.read_csv("Data/small_connections_matrix.csv", header=None)
conn_tier_mat = pd.read_csv("Data/connections_tier_matrix.csv", header=None)

# Filter to just be first num_flights rows and columns (plus remove the flight index row and column)
conn_mat = conn_mat.iloc[1:num_flights+1, 1:num_flights+1].to_numpy()  # Convert to numpy array
conn_tier_mat = conn_tier_mat.iloc[1:num_flights+1, 1:num_flights+1].to_numpy()

total_connection_wd_tier1 = 0.0
total_connection_passengers_tier1 = 0
total_connection_wd_tier2 = 0.0
total_connection_passengers_tier2 = 0
total_connection_wd_tier3 = 0.0
total_connection_passengers_tier3 = 0

# Load walking distances between gates.
# This CSV is expected to have a "Gate_Name" column and other columns with gate names.
walking_distances_gate_to_gate = pd.read_csv("Data/Walking Distances Gate-to-Gate.csv")

# Loop over flight pairs
num_rows, num_cols = conn_mat.shape
for i in range(0, num_rows):
    for j in range(0, num_cols):
        num_connect = conn_mat[i, j]
        if num_connect > 0:
            # Get tier of these connecting passengers
            tier = conn_tier_mat[i, j]

            # Retrieve flight rows corresponding to the current indices
            flight_i = df.iloc[i]
            flight_j = df.iloc[j]
            
            # Determine gate assignment based on flight type
            gate_i = flight_i["OptArrGate"] # Flight i is always inbound
            gate_i_int_match = walking_distances[walking_distances["Gate_Name"] == gate_i]
            gate_i = gate_i_int_match.iloc[0]["Gate_Int"]

            gate_j = flight_j["OptDepGate"] # Flight j is always outbound
            gate_j_int_match = walking_distances[walking_distances["Gate_Name"] == gate_j]
            gate_j = gate_j_int_match.iloc[0]["Gate_Int"]
            
            gate_to_gate_wd = walking_distances_gate_to_gate.iloc[gate_i-1, gate_j-1]

            if tier == 1:
                total_connection_wd_tier1 += num_connect * gate_to_gate_wd
                total_connection_passengers_tier1 += num_connect
            
            if tier == 2:
                total_connection_wd_tier2 += num_connect * gate_to_gate_wd
                total_connection_passengers_tier2 += num_connect

            if tier == 3:
                total_connection_wd_tier3 += num_connect * gate_to_gate_wd
                total_connection_passengers_tier3 += num_connect

avg_connection_wd_tier1 = total_connection_wd_tier1 / total_connection_passengers_tier1
avg_connection_wd_tier2 = total_connection_wd_tier2 / total_connection_passengers_tier2
avg_connection_wd_tier3 = total_connection_wd_tier3 / total_connection_passengers_tier3

print("Average Departing Passenger Walking Distance:", round(avg_departing_wd, 2))
print("Average Arriving Passenger Walking Distance:", round(avg_arriving_wd, 2))
print("Average Tier 1 Connecting Passenger Walking Distance:", round(avg_connection_wd_tier1, 2))
print("Average Tier 2 Connecting Passenger Walking Distance:", round(avg_connection_wd_tier2, 2))
print("Average Tier 3 Connecting Passenger Walking Distance:", round(avg_connection_wd_tier3, 2))
print("Total flights:", num_flights)
print("Total Departing Passengers:", total_departing_passengers)
print("Total Arriving Passengers:", total_arriving_passengers)
print("Total Tier 1 Connecting Passengers:", total_connection_passengers_tier1)
print("Total Tier 2 Connecting Passengers:", total_connection_passengers_tier2)
print("Total Tier 3 Connecting Passengers:", total_connection_passengers_tier3)

Metrics based on tiered optimized gate assignments:

Average Departing Passenger Walking Distance: 142.81
Average Arriving Passenger Walking Distance: 266.59
Average Tier 1 Connecting Passenger Walking Distance: 165.0
Average Tier 2 Connecting Passenger Walking Distance: 23.0
Average Tier 3 Connecting Passenger Walking Distance: 37.74
Total flights: 30
Total Departing Passengers: 3018
Total Arriving Passengers: 405
Total Tier 1 Connecting Passengers: 1
Total Tier 2 Connecting Passengers: 24
Total Tier 3 Connecting Passengers: 58


/var/folders/6f/0t_pqxjx06q6_18yc4lp8j3w0000gn/T/ipykernel_1369/3845687294.py:44: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  conn_tier_mat = pd.read_csv("Data/connections_tier_matrix.csv", header=None)
